<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/machine_learning/reinforcement_learning/OpenAI_gym/card_game_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Card Game Env: Własne środowisko Pythona

In [ ]:
try:
  %tensorflow_version 2.x
except:
  pass

TensorFlow 2.x selected.


In [ ]:
!pip install tf-agents
!pip install 'gym==0.10.11'

In [ ]:
from __future__ import absolute_import, division, print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

### Tworzenie własnego środowiska Pythona
Załóżmy, że chcemy przetrenować agenta do poniżeszej gry karcianej (inspirowanej Black Jackiem):

1.   W grze używa się skończonej talii kart ponumerowanych 1...10.
2.   W każdej kolejce agent musi zrobić jedną z 2 rzeczy: wziąć losową kartę albo opuścić kolejkę.
3.   Celem gry jest uzyskanie sumy kart tak bardzo jak top możliwe zbliżonej do 21 na końcu każdej rundy, bez przekraczania.

Środowisko przedstawiające grę mogłoby wyglądać tak:

1.   Akcje: Mamy 2 akcje. Akcja 0: weź nową kartę i Akcja 1: opuść kolejkę.
2.   Obserwacje: Suma kart w bieżącej rundzie.
3.   Nagroda: `sum_of_cards - 21 if sum_of_cards <= 21, else -21`







In [ ]:
class CardGameEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._state = 0
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      return self.reset()

    if action == 1:
      self._episode_ended = True
    elif action == 0:
      new_card = np.random.randint(1, 11)
      self._state += new_card
    else:
      raise ValueError('`akcja` powinna być 0 lub 1.')

    if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

In [ ]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

In [ ]:
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype=np.int32)

environment = CardGameEnv()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(1):
  time_step = environment.step(get_new_card_action)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(end_round_action)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([4], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(-17., dtype=float32), discount=array(0., dtype=float32), observation=array([4], dtype=int32))
Final Reward =  -17.0
